<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/temporal-analysis/completely_home/analyse_LA_mobility_poverty_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [2]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 171 µs (started: 2021-04-21 23:17:57 +00:00)


In [3]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 66kB/s 
     |████████████████████████████████| 204kB 42.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=959a35aef664a19c169eee275ff72014118cbbc3182d7eb08803495e6be4b8d8
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [4]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 99 ms (started: 2021-04-21 23:18:50 +00:00)


In [5]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 311 ms (started: 2021-04-21 23:18:50 +00:00)


In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.76 s (started: 2021-04-21 23:18:50 +00:00)


In [7]:
#sc.stop()

time: 3.71 ms (started: 2021-04-21 23:18:55 +00:00)


In [8]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 16.7 s (started: 2021-04-21 23:18:55 +00:00)


In [9]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'la'

time: 3.62 ms (started: 2021-04-21 23:19:12 +00:00)


In [10]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [11]:
df_soc.createOrReplaceTempView('clean_la')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count FROM clean_la WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+
|         cbg|                date|device_count|completely_home_device_count|
+------------+--------------------+------------+----------------------------+
|060371831013|2020-02-07T00:00:...|          45|                           6|
|060374034022|2020-02-07T00:00:...|         155|                          24|
|060375326041|2020-02-07T00:00:...|          31|                           4|
|060375544031|2020-02-07T00:00:...|         130|                          27|
|060375718003|2020-02-07T00:00:...|          35|                          10|
|060590018024|2020-02-07T00:00:...|          38|                          13|
|060590320463|2020-02-07T00:00:...|         184|                          24|
|060590626281|2020-02-07T00:00:...|         108|                          32|
|060371374011|2020-02-07T00:00:...|          72|                          30|
|060372398012|2020-02-07T00:00:...|          41|                

In [12]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 52.5 ms (started: 2021-04-21 23:19:38 +00:00)


In [13]:
df_soc.show()

+------------+----------+------------+----------------------------+
|         cbg|      date|device_count|completely_home_device_count|
+------------+----------+------------+----------------------------+
|060371831013|2020-02-07|          45|                           6|
|060374034022|2020-02-07|         155|                          24|
|060375326041|2020-02-07|          31|                           4|
|060375544031|2020-02-07|         130|                          27|
|060375718003|2020-02-07|          35|                          10|
|060590018024|2020-02-07|          38|                          13|
|060590320463|2020-02-07|         184|                          24|
|060590626281|2020-02-07|         108|                          32|
|060371374011|2020-02-07|          72|                          30|
|060372398012|2020-02-07|          41|                          15|
|060373009011|2020-02-07|          82|                          20|
|060375418021|2020-02-07|          49|          

In [14]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 68.7 ms (started: 2021-04-21 23:19:39 +00:00)


In [15]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 32.3 ms (started: 2021-04-21 23:19:39 +00:00)




---


**Demographic data**





---



In [16]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'la.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|060373104003|     1286|17.884914463452567| 69.51788491446345|
|060590865023|     2545|22.789783889980352| 85.34381139489194|
|060376510024|     1285|               0.0|42.490272373540854|
|060376511011|     2708| 7.745398773006134| 50.70162481536189|
|060590994023|      672|12.969283276450511| 38.54166666666667|
|060590995094|      342| 17.83625730994152| 77.48538011695906|
|060376511012|     2483|2.6178010471204187|50.302053966975436|
|060376512011|      859|2.8846153846153846| 82.42142025611176|
|060376512012|     1179| 1.441899915182358|  74.1306191687871|
|060376512014|      989|2.4266936299292214| 70.57633973710819|
|060376512211|     3373|  9.95684340320592| 40.61666172546695|
|060590995092|      938| 5.756929637526652|  84.9680170575693|
|060590995095|      576|14.583333333333334|           7

In [17]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+----------------------------+--------------------------+------------------+
|         cbg|      date|device_count|completely_home_device_count|completely_home_percentage|poverty_percentage|
+------------+----------+------------+----------------------------+--------------------------+------------------+
|060371831013|2020-02-07|          45|                           6|        13.333333333333334|15.450643776824036|
|060374034022|2020-02-07|         155|                          24|        15.483870967741936| 10.67193675889328|
|060375326041|2020-02-07|          31|                           4|        12.903225806451612| 27.79187817258883|
|060375544031|2020-02-07|         130|                          27|         20.76923076923077|  25.3447633246366|
|060375718003|2020-02-07|          35|                          10|         28.57142857142857|3.1537450722733245|
|060590018024|2020-02-07|          38|                          13|         34.210526315

In [18]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.completely_home_percentage as top20_perc,
l.completely_home_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 340 ms (started: 2021-04-21 23:19:41 +00:00)


**Load 2019 stats**

In [50]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][5]


+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|  _c0|             total|  completely_home|   part_time_work|   full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|             month|
+-----+------------------+-----------------+-----------------+-----------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+------------------+
|count|         3004769.0|        3004769.0|        3004769.0|        3004769.0|                 3004769.0|                3004769.0|                3004769.0|           3004769.0|           3004769.0|           3004769.0|         3004769.0|
| mean| 87.84796368705881|27.763

In [51]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 39.4 ms (started: 2021-04-22 00:16:17 +00:00)


In [21]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 26.3 ms (started: 2021-04-21 23:19:41 +00:00)



**Visualizations** 



In [61]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': ['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 8.61 ms (started: 2021-04-22 00:37:20 +00:00)


In [58]:
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','completely_home_value']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('completely_home_value:Q', title = 'Completely Home %'),
      color='Poverty %:N'
    ).properties(width = 1000)

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 11.6 ms (started: 2021-04-22 00:35:09 +00:00)


In [24]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 4.39 ms (started: 2021-04-21 23:19:41 +00:00)


In [25]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,17.297551,17.028372,-12.847941,-13.117120
1,2020-04-13,49.179507,27.580760,19.034015,-2.564731
2,2020-06-24,33.002194,36.514949,2.856702,6.369457
3,2020-06-08,36.723946,40.887284,6.578454,10.741793
4,2020-09-12,33.388659,29.334402,3.243167,-0.811090
...,...,...,...,...,...
361,2020-04-05,51.744280,27.824742,21.598788,-2.320750
362,2020-05-01,43.087875,43.937332,12.942383,13.791840
363,2020-10-25,35.361961,32.028754,5.216469,1.883262
364,2020-12-28,41.489642,24.049980,11.344151,-6.095512


time: 4min 40s (started: 2021-04-21 23:19:41 +00:00)


In [26]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 4.8 ms (started: 2021-04-21 23:24:22 +00:00)


In [62]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','completely home %']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('completely home %:Q', title = 'Completely Home %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 59.2 ms (started: 2021-04-22 00:37:27 +00:00)


In [28]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 14.7 ms (started: 2021-04-21 23:24:22 +00:00)


Filtering based on important dates



*   Thursday, March 19, 2020: Statewide Stay-at-Home Order Issued











In [63]:
start_date = '2020-3-16'
end_date = '2020-4-9'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 56 ms (started: 2021-04-22 00:37:55 +00:00)





*   August. 7, 2020: LA County Exceeds 200,000 Coronavirus Cases
*   August, 12, 2020: LA County Crosses 'Tragic Milestone' Of 5,000 Coronavirus Deaths 









In [64]:
start_date = '2020-8-5'
end_date = '2020-8-21'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 52.3 ms (started: 2021-04-22 00:38:15 +00:00)


*   Oct. 14, 2020: LA County Sees Rise In Workplace Outbreaks As Infection Rate Creeps Upward





In [65]:
start_date = '2020-10-12'
end_date = '2020-10-24'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 47.2 ms (started: 2021-04-22 00:38:36 +00:00)


Dec. 16, 2020:
LA County COVID-19 Deaths Hit New Record


In [66]:
start_date = '2020-12-15'
end_date = '2020-12-31'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 60.2 ms (started: 2021-04-22 00:38:40 +00:00)
